# Train with ADE20k via Gluon the LUCAS IMAGES
We train on the 140 images selected where the LC is represented and inference the semantic segmentation objects for those images with the ADE20K dataset


In [36]:
import pandas as pd
import numpy as np
import shutil

from PIL import Image 
import PIL 
import cv2

import os
from pathlib import Path
import glob
import sys
sys.path.append("./git")

import RF_utils as utils

import mxnet as mx
from mxnet import image
from mxnet.gluon.data.vision import transforms
import gluoncv
from gluoncv.data.transforms.presets.segmentation import test_transform
from gluoncv.utils.viz import get_color_pallete

Set the paths for the code


In [2]:
project_path = '/eos/jeodpp/data/projects/REFOCUS/data'

In [3]:
images_csv_path = '{}/LandCOverCLass/inputs/'.format(project_path)
inputs_path = '{}/LandCOverCLass/inputs/dataset/'.format(project_path)
out_path = '{}/LandCOverCLass/outputs/'.format(project_path)

Now we use the LUCAS Harmonized Database, see: https://www.nature.com/articles/s41597-020-00675-z

In [4]:
lucas_harmo = '{}/lucas_harmo.csv'.format(images_csv_path)
gt = pd.read_csv(lucas_harmo)
gt = gt[gt['year'] == 2018]

/eos/jeodpp/data/projects/REFOCUS/data/LandCOverCLass/venv/mxnet_gluon/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (6,13,28,29,30,33,34,35,38,39,40,41,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,67,68,69,70,71,81,83,84,85,86,87,88,89,93,95,97,99,101,102,103,104,105,106,107,108,109,112,113,114,115,116) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Select the list of images and find all the atribute of those images in the LUCAS database 2018

In [5]:
#fetch the images
imgs = glob.glob('{}*jpg'.format(inputs_path))

#finsd the images in the LUCAS database
limgs = [int(im.split('P')[0].split('/')[-1]) for im in imgs]
df = pd.DataFrame(limgs, columns = ['id'])
ids = gt['point_id'].tolist()
boolean_series = df.id.isin(ids)
df = df[boolean_series]
df['lc1'] = df.id.map(gt.set_index('point_id')['letter_group'])

#Save the filtered table wher we have the id of the images and the LC associated to it 
df.to_csv('{}Final-dataset-140xclass.csv'.format(images_csv_path))

## Inference of with Deeplabv3 +

We use the one implemented by Gluon and trained with ADE20k dataset

see: https://github.com/dmlc/gluon-cv

In [6]:
# Use the CPU
ctx = mx.cpu()

# get the labels from ADE20K
labels = utils.retrieve_ADE()

# select the backbones that we want to try during inference:
models = ['deeplab_resnest50_ade', 'deeplab_resnest101_ade', 'deeplab_resnest200_ade', 'deeplab_resnest269_ade']

images = df.copy()

In [ ]:
# Loop thourg the available backbones and do the inference for each of them

for model_name in models:
    
    #crate the dataframe to gather all the data from the inference
    name_csv = '{}test-{}.csv'.format(out_path, model_name)
    
    ade20kdata = utils.create_dataframe(labels, images)
    
    for inx, row in images.iterrows():
        print(row['id'])
      # Check if the image is already procesed
        if len(ade20kdata.loc[ade20kdata['id'] == row['id']]) == 0:  
            aux = '{}{}P.jpg'.format(inputs_path, row['id'])
            img = image.imread(aux)

            #normalize the image using mean
            img = test_transform(img, ctx)

            #get the backbone to use
            model = gluoncv.model_zoo.get_model(model_name, pretrained=True)
            
            #Inference on the image
            output = model.predict(img)
            predict = mx.nd.squeeze(mx.nd.argmax(output, 1)).asnumpy()
            
            #save the image
            Path("{}/{}/masks/".format(out_path, model_name,)).mkdir(parents=True, exist_ok=True)
            cv2.imwrite("{}/{}/masks/{}P.png".format(out_path, model_name, row['id']), predict)


            #get values of ade20k
            mask = get_color_pallete(predict, 'ade20k')
            
            #Get the pixel "area" for each ADE20K class predicted
            (unique, counts) = np.unique(mask, return_counts=True)
            
            #add data to the dataframe
            ade20kdata.loc[inx, 'id'] = row['id']
            ade20kdata.loc[inx, 'lc'] = row['lc1']
              
            for i in range(len(unique)):
                clas = labels[unique[i]]
                ade20kdata.loc[inx, str(clas)] = counts[i]
            
            ade20kdata.to_csv(name_csv)

        else:
            print('Image: {} already processed'.format(row['id']))



26581966
26642010
26881760


In [25]:

ade20kdata["id"][inx] = row['id']
ade20kdata["lc"][inx] = row['lc1']

In [35]:
cv2.imwrite("{}/{}/masks/{}P.png".format(out_path, model_name, row['id']), predict)

False

In [32]:
ade20kdata

,wall,building,sky,floor,tree,ceiling,road,bed,window,grass,...,plate,monitor,notice board,shower,radiator,glass,clock,flag,id,lc
0,0,5773,0,0,0,256,0,921,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,4920,1312,255202,0,61139,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,11921,5000,102146,0,251283,0,0,0,0,...,0,0,0,0,0,0,0,0,0,C
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1108,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1110,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
            ade20kdata["id"][inx] = row['id']
            ade20kdata["lc"][inx] = row['lc1']